In [68]:
import csv
import pdfplumber
import re

with pdfplumber.open("Participants.pdf") as pdf:
    participants = []
    is_participant_section = False
    page_number = 1

    for page in pdf.pages:
        text = page.extract_text()
        lines = text.split('\n')

        print(f"Processing Page {page_number}")
        for line in lines:
            if "Участники" in line or "участники" in line:
                is_participant_section = True

            if is_participant_section:
                line_cleaned = re.sub(r'^\d+\.\s*', '', line).strip()  
                line_cleaned = re.sub(r'\d+', '', line_cleaned).strip()  

                if re.search(r'\d', line_cleaned):
                    line_cleaned = re.sub(r'(\S+\s\S+)(\d+)', r'\1\n\2', line_cleaned)

                match = re.match(r"^(.*\S.*)$", line_cleaned)
                if match:
                    participant_name = match.group(1).strip()
                    participants.append(participant_name)

                    print(f"Oчищенная строка: '{line_cleaned}'")
                    print(f"Добавлен участник: '{participant_name}'")

        with open("participants.csv", mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            if page_number == 1:
                writer.writerow(["Категория", "Имя"])  

            for participant in participants:
                writer.writerow(["Участники", participant])

        print(f"Загружены участники страницы {page_number}:")
        for participant in participants:
            print(f"- {participant}")

        participants.clear()
        page_number += 1

Processing Page 1
Oчищенная строка: 'Участники (студенты)'
Добавлен участник: 'Участники (студенты)'
Oчищенная строка: 'Карачков Владислав'
Добавлен участник: 'Карачков Владислав'
Oчищенная строка: 'Нурлан Айдын'
Добавлен участник: 'Нурлан Айдын'
Oчищенная строка: 'Жолтаев Темірлан'
Добавлен участник: 'Жолтаев Темірлан'
Oчищенная строка: 'Тугелбаев Галымжан'
Добавлен участник: 'Тугелбаев Галымжан'
Oчищенная строка: 'Amir Kairov'
Добавлен участник: 'Amir Kairov'
Oчищенная строка: 'Демубай Қадыр Құрбанұлы'
Добавлен участник: 'Демубай Қадыр Құрбанұлы'
Oчищенная строка: 'Rakhmanberdiyeva Nurbibi'
Добавлен участник: 'Rakhmanberdiyeva Nurbibi'
Oчищенная строка: 'Karzhaubayev Maxat'
Добавлен участник: 'Karzhaubayev Maxat'
Oчищенная строка: 'Bakytbekov Ulan'
Добавлен участник: 'Bakytbekov Ulan'
Oчищенная строка: 'Myrzabek Yedil'
Добавлен участник: 'Myrzabek Yedil'
Oчищенная строка: 'Рахметова Улдана Рахатовна'
Добавлен участник: 'Рахметова Улдана Рахатовна'
Oчищенная строка: 'Сарсенбаева Адия 

In [72]:
import pandas as pd

df = pd.read_csv('participants.csv')

In [73]:
df

,Категория,Имя
0,Участники,Участники (студенты)
1,Участники,Карачков Владислав
2,Участники,Нурлан Айдын
3,Участники,Жолтаев Темірлан
4,Участники,Тугелбаев Галымжан
...,...,...
396,Участники,Жуаныш Аружан
397,Участники,Нурлыбаева Камила
398,Участники,Мулкуланова Лада
399,Участники,Қабдолла Риза


In [1]:
import os
from PIL import Image, ImageDraw, ImageFont

def get_font_size(text, font_path, max_width, max_height):
    font_size = 250  
    font = ImageFont.truetype(font_path, font_size)
    
    bbox = font.getbbox(text)
    width, height = bbox[2] - bbox[0], bbox[3] - bbox[1]
    
    while width > max_width or height > max_height:
        font_size -= 10
        font = ImageFont.truetype(font_path, font_size)
        bbox = font.getbbox(text)
        width, height = bbox[2] - bbox[0], bbox[3] - bbox[1]
    
    return font

if not os.path.exists("Certificates"):
    os.makedirs("Certificates")

template = "datathon.png"  

img = Image.open(template)

draw = ImageDraw.Draw(img)

font_text = ImageFont.truetype("arial.ttf", 150)

max_name_width = 1500  
max_name_height = 300  

with open('participants.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file) 

    for index, row in enumerate(csv_reader, start=2): 
        name = row['Имя'] 

        font_name = get_font_size(name, "arial.ttf", max_name_width, max_name_height)

        img_copy = img.copy()
        draw_copy = ImageDraw.Draw(img_copy)

        x_name, y_name = 450, 1300  

        bbox = draw_copy.textbbox((x_name, y_name), name, font=font_name)
        width, height = bbox[2] - bbox[0], bbox[3] - bbox[1]

        if width > max_name_width:
            y_name += 100  

        draw_copy.text((x_name, y_name), name, (0, 0, 0), font=font_name)

        text = "Участник DATATHON 2024"
        x_text, y_text = 450, y_name + height + 400  

        if width > max_name_width:
            y_text += 100  

        draw_copy.text((x_text, y_text), text, (0, 0, 0), font=font_text)

        certificate_filename = f"Certificates/certificate_{index-2}.png"
        img_copy.save(certificate_filename)